this notebook will be used to show the performance of the first attempt at learning reward.

first load the trained reward network anbd setup methods.

In [1]:
from baselines.common.vec_env import VecFrameStack
from comp300.LearningModel.AgentClasses import *
from baselines.common.cmd_util import make_vec_env
from gym import register
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np

from os import listdir
from os.path import isfile, join
import re

/home/patrick/PycharmProjects/comp300/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/patrick/PycharmProjects/comp300/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/patrick/PycharmProjects/comp300/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/patrick/PycharmP

In [2]:


#setup the env
register(id='ChessSelf-v0',
             entry_point='comp300.Chess.ChessWrapper:ChessEnv',
             max_episode_steps=1000)

env_id = 'ChessSelf-v0'
env_type = 'ChessWrapper'

env = make_vec_env(env_id, env_type, 1, 0,
                   flatten_dict_observations=True,
                   wrapper_kwargs={
                       'clip_rewards': False,
                       'episode_life': False,
                   })


#load the reward network
trainedNetwork = RewardNetwork("", env_type, env=env)

trainedNetwork.load_state_dict(torch.load("/home/patrick/models/chess-reward/fulltest6.params"))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
trainedNetwork.to(device)


Logging to /tmp/openai-2020-03-25-13-34-41-817013



RewardNetwork(
  (fc1): Linear(in_features=66, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)

now load all models and run each to get demos to run the network on

In [4]:
trueRewards = []
learnedRewards = []

from comp300.Chess.LearnRewardChess import generate_chess_demos
demos, trueRewards = generate_chess_demos(env, 200)

for i in range(len(demos)):
    shapedObservation = np.array(demos[i])
    reward, abs_reward = trainedNetwork.predict_reward(shapedObservation)
    learnedReward += reward.tolist()
    learnedRewards.append(learnedReward)
    



  0:finished demo with skill level 20, Result = [100.]
  1:finished demo with skill level 7, Result = [100.]
  2:finished demo with skill level 20, Result = [-100.]
  3:finished demo with skill level 13, Result = [100.]
  4:finished demo with skill level 10, Result = [100.]
  5:finished demo with skill level 11, Result = [100.]
  6:finished demo with skill level 2, Result = [100.]
  7:finished demo with skill level 8, Result = [100.]
  8:finished demo with skill level 14, Result = [100.]
  9:finished demo with skill level 13, Result = [100.]
  10:finished demo with skill level 7, Result = [-100.]
  11:finished demo with skill level 16, Result = [100.]
  12:finished demo with skill level 14, Result = [100.]
  13:finished demo with skill level 6, Result = [100.]
  14:finished demo with skill level 14, Result = [-100.]
  15:finished demo with skill level 6, Result = [-100.]
  16:finished demo with skill level 20, Result = [100.]
  17:finished demo with skill level 13, Result = [-100.]
  1

  147:finished demo with skill level 13, Result = [-100.]
  148:finished demo with skill level 4, Result = [100.]
  149:finished demo with skill level 10, Result = [100.]
  150:finished demo with skill level 20, Result = [100.]
  151:finished demo with skill level 2, Result = [100.]
  152:finished demo with skill level 4, Result = [100.]
  153:finished demo with skill level 16, Result = [100.]
  154:finished demo with skill level 12, Result = [-100.]
  155:finished demo with skill level 15, Result = [100.]
  156:finished demo with skill level 9, Result = [100.]
  157:finished demo with skill level 1, Result = [100.]
  158:finished demo with skill level 14, Result = [100.]
  159:finished demo with skill level 12, Result = [100.]
  160:finished demo with skill level 1, Result = [-100.]
  161:finished demo with skill level 1, Result = [-100.]
  162:finished demo with skill level 6, Result = [0.]
  163:finished demo with skill level 8, Result = [100.]
  164:finished demo with skill level 1

ValueError: Type must be a sub-type of ndarray type

In [ ]:

minLearned = min(learnedRewards)
normalisedRewards = [x-minLearned for x in learnedRewards]
copyLearned = []
copyTrue = []
for i in range(len(normalisedRewards)):
    if normalisedRewards[i] > 500000 or trueRewards[i] < -100:
        pass
    else:
        copyLearned.append(normalisedRewards[i])
        copyTrue.append(trueRewards[i])

minLearned = min(copyLearned)
copyLearned = [x-minLearned for x in copyLearned]

minTrue = min(copyTrue)
copyTrue = [x-minTrue for x in copyTrue] 
maxTrue = max(copyTrue)

maxLearned = max(copyLearned)
copyLearned = (copyLearned) / (maxLearned / maxTrue)

In [ ]:
from matplotlib.pyplot import figure

print("{},{}".format(maxTrue, maxLearned))

figure(num=None, figsize=(10, 7), dpi=80, facecolor='w', edgecolor='k')
plt.scatter(np.array(copyTrue), np.array(copyLearned), c='r')
plt.plot(np.arange(100), np.arange(100))
plt.ylabel("Learned reward")
plt.xlabel("Ground truth reward")
plt.title("graph of learned reward against ground truth")
plt.show()

In [ ]:
minReward = [min(trueRewards)]
maxReward = [max(trueRewards)]
average = [sum(trueRewards) / len(trueRewards)]

from comp300.LearningModel.getAverageReward import *
agent.load("~/models/ant-reward-rl/ant_2M_ppo2")
meanR, minR,maxR, std = getAvgReward(agent, env, 20)

minReward.append(minR)
maxReward.append(maxR)
average.append(meanR)


In [ ]:
minReward[0] = min(trueRewards)
maxReward[0] = max(trueRewards)
average[0] = sum(trueRewards) /len(trueRewards)
print("mins: {}, maxs: {}, means: {}".format(minReward, maxReward, average))
# create plot
figure(num=None, figsize=(10, 7), dpi=80, facecolor='w', edgecolor='k')
#fig, ax = plt.subplots()
index = np.arange(2)
bar_width = 0.3
opacity = 0.8

'''
rects1 = plt.bar(index, minReward, bar_width,
alpha=opacity,
color='b',
label='Minimum Reward')
'''

rects2 = plt.bar(index + bar_width, average, bar_width,
alpha=opacity,
color='g',
label='Average Reward')

rects3 = plt.bar(index + bar_width +bar_width, maxReward, bar_width,
alpha=opacity,
color='r',
label='Max Reward')

plt.xlabel('Agent')
plt.ylabel('Reward')
plt.title('The min, max and mean reward of the demonstrator and trained agent')
plt.xticks(index + bar_width, ('Demonstrations', 'Trained agent'))
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from baselines.common import plot_util as pu
results = pu.load_results('~/logs/Agent/mspacman')
pu.plot_results(results, average_group=True, split_fn=lambda _: '', figsize=(10, 7))